In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import torchaudio

/home/akansh/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [3]:
from tqdm import tqdm

In [4]:
import matplotlib.pyplot as plt

In [5]:
import librosa
import librosa.display

In [6]:
# os.makedirs('Spectrogram(Sets)')

### Slicing the audio

In [ ]:
def labels_eko(wav_file_name):
    k = wav_file_name
    b = k.split('_')
    b[1] = 'Label'
    separator = '_'
    return separator.join(b)[:-3] +'txt'

### Eko

In [ ]:
set_loc = "./Sets/"
data_loc = "./"
new_sample_rate = 16000
channel = 0
os.makedirs('./Sliced_eko(Sets)')
for i in os.listdir(set_loc):
    os.makedirs("./Sliced_eko(Sets)/"+i[:-4])
    os.makedirs("./Sliced_eko(Sets)/"+i[:-4]+'/Asthma')
    os.makedirs("./Sliced_eko(Sets)/"+i[:-4]+'/Normal')
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        for k in os.listdir(data_loc+j+'/Eko'):
            waveform_eko, sample_rate = torchaudio.load(data_loc+j+'/Eko/'+k)
            txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    A = waveform_eko[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))]
                    if j.find("sthma") != -1:
                        torchaudio.save("./Sliced_eko(Sets)/"+i[:-4]+'/Asthma/'+k[:-4]+'_'+ str(l)+'.wav', A, sample_rate)
                    else:
                        torchaudio.save("./Sliced_eko(Sets)/"+i[:-4]+'/Normal/'+k[:-4]+'_'+ str(l)+'.wav', A, sample_rate)
                        
                    

In [ ]:
2269+2574

### Mic

In [ ]:
set_loc = "./Sets/"
data_loc = "./"
new_sample_rate = 16000
channel = 0
os.makedirs('./Sliced_mic(Sets)')
for i in os.listdir(set_loc):
    os.makedirs("./Sliced_mic(Sets)/"+i[:-4])
    os.makedirs("./Sliced_mic(Sets)/"+i[:-4]+'/Asthma')
    os.makedirs("./Sliced_mic(Sets)/"+i[:-4]+'/Normal')
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        for k in os.listdir(data_loc+j+'/Mic'):
            waveform_mic, sample_rate = torchaudio.load(data_loc+j+'/Mic/'+k)
            waveform_mic = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform_mic[channel,:].view(1,-1))
            sample_rate = new_sample_rate
            txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    A = waveform_mic[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))]
                    if j.find("sthma") != -1:
                        torchaudio.save("./Sliced_mic(Sets)/"+i[:-4]+'/Asthma/'+k[:-4]+'_'+ str(l)+'.wav', A, sample_rate)
                    else:
                        torchaudio.save("./Sliced_mic(Sets)/"+i[:-4]+'/Normal/'+k[:-4]+'_'+ str(l)+'.wav', A, sample_rate)
                        
                    

### Creating Spectrogram

In [6]:
for i in os.listdir("./Sliced_mic(Sets)/"):
    try: 
        os.makedirs("./Sliced_mic(Sets)/"+i+'/Spectrogram')
        os.makedirs("./Sliced_mic(Sets)/"+i+'/Spectrogram/'+'Normal')
        os.makedirs("./Sliced_mic(Sets)/"+i+'/Spectrogram/'+'Asthma')

        for k in tqdm(os.listdir("./Sliced_mic(Sets)/"+i+'/Asthma')):
            waveform, sr = librosa.load("./Sliced_mic(Sets)/"+i+'/Asthma/'+k, sr = None)
            n_fft = int(20*sr*(2*10**-3))
            window = int(n_fft/2)
            hop_length = int(window/2)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform, n_fft = n_fft,win_length = window, hop_length = hop_length)))
            # Plotting the spectrogram and save as JPG without axes (just the image)
            fig = plt.figure(figsize=[1,1])
            ax = fig.add_subplot(111)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            ax.set_frame_on(False)
            librosa.display.specshow(D,x_axis='time', y_axis='hz')
            plt.savefig("./Sliced_mic(Sets)/"+i+'/Spectrogram/'+'Asthma/'+k[:-4]+'.jpg',dpi=400,bbox_inches = 'tight', frameon='false', pad_inches=0)
            plt.close()

        for k in tqdm(os.listdir("./Sliced_mic(Sets)/"+i+'/Normal')):
            waveform, sr = librosa.load("./Sliced_mic(Sets)/"+i+'/Normal/'+k, sr = None)
            n_fft = int(20*sr*(2*10**-3))
            window = int(n_fft/2)
            hop_length = int(window/2)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform, n_fft = n_fft,win_length = window, hop_length = hop_length)))
            # Plotting the spectrogram and save as JPG without axes (just the image)
            fig = plt.figure(figsize=[1,1])
            ax = fig.add_subplot(111)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            ax.set_frame_on(False)
            librosa.display.specshow(D,x_axis='time', y_axis='hz')
            plt.savefig("./Sliced_mic(Sets)/"+i+'/Spectrogram/'+'Normal/'+k[:-4]+'.jpg',dpi=400,bbox_inches = 'tight', frameon='false', pad_inches=0)
            plt.close()
    except:
        pass

        


100%|██████████| 293/293 [00:43<00:00,  6.64it/s]


In [ ]:
for i in os.listdir("./Sliced_eko(Sets)/"):
    try: 
        os.makedirs("./Sliced_eko(Sets)/"+i+'/Spectrogram')
        os.makedirs("./Sliced_eko(Sets)/"+i+'/Spectrogram/'+'Normal')
        os.makedirs("./Sliced_eko(Sets)/"+i+'/Spectrogram/'+'Asthma')

        for k in tqdm(os.listdir("./Sliced_eko(Sets)/"+i+'/Asthma')):
            waveform, sr = librosa.load("./Sliced_eko(Sets)/"+i+'/Asthma/'+k, sr = None)
            n_fft = int(20*sr*(2*10**-3))
            window = int(n_fft/2)
            hop_length = int(window/2)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform, n_fft = n_fft,win_length = window, hop_length = hop_length)))
            # Plotting the spectrogram and save as JPG without axes (just the image)
            fig = plt.figure(figsize=[1,1])
            ax = fig.add_subplot(111)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            ax.set_frame_on(False)
            librosa.display.specshow(D,x_axis='time', y_axis='hz')
            plt.savefig("./Sliced_eko(Sets)/"+i+'/Spectrogram/'+'Asthma/'+k[:-4]+'.jpg',dpi=400,bbox_inches = 'tight', frameon='false', pad_inches=0)
            plt.close()

        for k in tqdm(os.listdir("./Sliced_eko(Sets)/"+i+'/Normal')):
            waveform, sr = librosa.load("./Sliced_eko(Sets)/"+i+'/Normal/'+k, sr = None)
            n_fft = int(20*sr*(2*10**-3))
            window = int(n_fft/2)
            hop_length = int(window/2)
            D = librosa.amplitude_to_db(np.abs(librosa.stft(waveform, n_fft = n_fft,win_length = window, hop_length = hop_length)))
            # Plotting the spectrogram and save as JPG without axes (just the image)
            fig = plt.figure(figsize=[1,1])
            ax = fig.add_subplot(111)
            ax.axes.get_xaxis().set_visible(False)
            ax.axes.get_yaxis().set_visible(False)
            ax.set_frame_on(False)
            librosa.display.specshow(D,x_axis='time', y_axis='hz')
            plt.savefig("./Sliced_eko(Sets)/"+i+'/Spectrogram/'+'Normal/'+k[:-4]+'.jpg',dpi=400,bbox_inches = 'tight', frameon='false', pad_inches=0)
            plt.close()
    except:
        pass

        


 37%|███▋      | 141/378 [00:39<01:01,  3.84it/s]